In [ ]:
pip install fastai

In [7]:
pip install duckduckgo-search

In [ ]:
pip install fastcore

In [1]:
pip install imageio[pyav]

In [6]:
# Импортируем библиотеки
from fastai.vision.all import *
from duckduckgo_search import DDGS
import time 
import json
import os
import shutil
from imageio.v2 import imread
import datetime


In [8]:
# Считываем словарь птиц России из файла json
with open('russian_birds.json', 'r') as f:
    json_string=f.read()
    birds_dict = json.loads(json_string)

In [9]:
# Если хотим загрузить тоько часть видов птиц - создаем укороченный словарь видов птиц. 
# Если хотим делать датасет со всеми видами птиц - этот шаг пропускаем
Some_birds=('0','40','80','120','160','200','240','280','320','360','400','440','480','520','560','600','640','680','720','760','800') # делаем выборку только по 21 виду птиц, в качестве доказательсва работоспособности проекта. 
temp_dict={}
for i in Some_birds:
    temp_dict[i]=birds_dict[i]
# Сохраняем словарь выбранных птиц в 'russian_birds_1.json'
with open('russian_birds_1.json', 'w') as f:
    json.dump(temp_dict, f) # Записываем укороченный словарь в файл 'russian_birds_1.json'. Далее мы будем загружать данные из него
    
    



In [9]:
# Этот шаг делаем, если хотим загружать датасет по всем видам.
# Если хотим только конкретные виды птиц - этот шаг не делаем, а делаем предыдущий
with open('russian_birds_1.json', 'w') as f:
    json.dump(birds_dict, f) # Записываем полный словарь в файл 'russian_birds_1.json'. Далее мы будем загружать данные из него

In [ ]:
# Функция возвращающяа ссылки на картинки из поискового запроса. Поисковой запрос с поискового сервиса duckduckgo.com 
ddgs = DDGS()
def search_images(term, max_images=100):
    """
        Возвращает список URL адресов изображений результатов поискового запроса с поискового сервиса duckduckgo.com 
    Args:
        term (str): текст поискового запроса.
        max_images (int(optional)): Максимальное количество ответов. По умолчанию 1000
    
    Return:    
        Возвращяет список URL адресов изображений результатов поискового запроса
    """
    
    print(f"Searching for '{term}'")
    return L(ddgs.images(keywords=term, max_results=max_images)).itemgot('image')

In [14]:
def move_files(from_dir,to_dir,delite_from_after=True):
    """
        Переносит все фотографии из одной папки в другую. Опционально удаляет первую папку после переноса фотографий 
    Args:
        from_dir(str): папка из которой переносятся файлы,
        to_dir(str): папка в которую переносятся файлы,
        delite_from_after(Bool(optional)): флаг (нужно ли удалить исходную папку).
    """

    
    images_paths = get_image_files(from_dir)
    mist=0
    for val in images_paths:
       try:
            name=os.path.split(val)[-1]                                   # вычленяем из пути файла имя файла
            shutil.move(val, f'{to_dir}/{name}')
            
       except: 
            print(f"ошибка при переносе в папке {from_dir}")
            mist=1
    try:       
        if (delite_from_after==True) and  (mist==0):
        
            os.rmdir(from_dir)
                
    except: 
            print(f"ошибка при удалении папки {from_dir}")        
 

In [13]:
def rename_files_in_dir(data_dir,prefix):
    """
        Переименовывает файлы в папке с использованием префикса 
    Args:
        data_dir (str): Путь к папке в которой переименовываются все файлы. 
        prefix(str): префикс, который будет в названии файла,
     """
    
    images_paths = get_image_files(data_dir)
    for ind,val in enumerate(images_paths):
       try:
            ext=str(val).split('.')[-1]# Расширение файла
            os.rename(val, f'{data_dir}/{prefix}_00{ind}.{ext}')# переименовываем файл по схеме
       except: 
            print("отработал ошибку")
    
    

In [ ]:
# Функция удаляет нечитаемые картинки в папке
def verify_files(data_dir):
    """
        Проверяет все ли изображения в выбранной папке читаются. Если какие-то файлы с изображениями не читается - удаляет эти файлы
    Args:
        data_dir (str): Путь к директории с данными. 
    
    Return:    
        Возвращяет первоначальное количество файлов с изображениями и количество удаленных файлов
    """

    images=get_files(data_dir) # получаем перечень файлов
    failed = verify_images(images) # проверяем файлы и получаем перечень нечитаемых
    failed.map(Path.unlink) # удаляем нечитаемые файлы 
    return(len(images), len(failed))

In [ ]:
# Основной Код
path='D:/Russian_birds_selected'
kirilic='ёйцукенгшщзхъфывапролджэячсмитьбю'
log = 'log__1.txt'

start = time.time() ## точка отсчета времени общего старта
with open('russian_birds_1.json', 'r') as f:
    birds_dict = json.loads(f.read()) #загружаем словарь видов птиц
    birds_dict_copy = birds_dict.copy() #создаем копию словаря видов птиц

with open(log, 'a+') as f: # записываем строку в log файл о старте загрузки
    f.write(f'-----------------------Start at: {datetime.datetime.now()} ------------------------------ \n')

for bird_class in birds_dict: # Проходим по ключам словаря видов птиц. Ключ словаря будем использовать, как название папки для фотографий одного вида птиц.
    print(f'{bird_class}  {birds_dict[bird_class]}')
    
    with open(log, 'a+') as f: # записываем строку в log файл о начале скачивания папки
        f.write(f'{bird_class}  {birds_dict[bird_class]}\n')

    start_dounload = time.time() # точка отсчета времени старта загрузки папки
    
    # Скачиваем файлы по запросам:
    os.makedirs(f'{path}/{bird_class}', exist_ok=True)# Создаем папку вида птиц
    
    # Дальше мы будем загружать фотографии по запросу с поисковика duckduckgo.com в несколько подходов:
    # 1)     10 фотографий с запросом: название по латыни + male (для того, чтобы у нас был референс, как выглядят самцы этого вида ).Грузим в отдельную подпапку. Позднее все подпапки сольем в одну. 
    # 2)     10 фотографий с запросом: название по латыни + female (для того, чтобы у нас был референс, как выглядят самки этого вида ).Грузим в отдельную подпапку. Позднее все подпапки сольем в одну. 
    # 2)     20 фотографий с запросом: название по латыни (для того, чтобы у нас был референс, как выглядят птицы этого вида ).Грузим в отдельную подпапку. Позднее все подпапки сольем в одну. 

    # 3)     1000 (фактически будет меньше +-250 - 300) фотографий с запросом: название по латыни + bird photo.Грузим в основную папку вида.
    # 4)     1000 (фактически будет меньше +-250 - 300) фотографий с запросом: название по русски + птица фото.Грузим в основную папку вида.


    # Грузим 10 фотографий с запросом: название по латыни + male. Переименовываем их и скидываем папку вида питц  
    dir=f'{path}/{bird_class}/male'
    to_dir=f'{path}/{bird_class}'
    os.makedirs(dir, exist_ok=True)# Создаем папку вида птиц/male
    download_images(dir, urls=search_images(f'{birds_dict[bird_class][-1]} male',10))# Грузим самцов, для референса
    species_name=birds_dict[bird_class][0]
    species_name=''.join(species_name.replace(' ','_').split(','))#удаляем запятые и заменяем пробеля на _
    prefix=f'00000_{species_name}_самец_'## ноли в начале, для того, чтобы в папке по алфавиту быть на верху
    rename_files_in_dir(dir,prefix)# переименовываем файлы в папке     
    move_files(dir,to_dir)# перемещаем файлы в родительскую папку и удаляем подпапку   
    time.sleep(10)  # Задержка на 10 секунд

    # Грузим 10 фотографий с запросом: название по латыни + female. Переименовываем их и скидываем папку вида питц  
    dir=f'{path}/{bird_class}/female'
    to_dir=f'{path}/{bird_class}'
    os.makedirs(dir, exist_ok=True)# Создаем папку вида птиц/male
    download_images(dir, urls=search_images(f'{birds_dict[bird_class][-1]} female',10))# Грузим самок, для референса
    species_name=birds_dict[bird_class][0]
    species_name=''.join(species_name.replace(' ','_').split(','))#удаляем запятые и заменяем пробеля на _
    prefix=f'00000_{species_name}_самка_'## ноли в начале, для того, чтобы в папке по алфавиту быть на верху
    rename_files_in_dir(dir,prefix)# переименовываем файлы в папке     
    move_files(dir,to_dir)# перемещаем файлы в родительскую папку и удаляем подпапку  
    time.sleep(10)  # Задержка на 10 секунд

    # Грузим 20 фотографий с запросом: название по латыни + female. Переименовываем их и скидываем папку вида питц  
    dir=f'{path}/{bird_class}/female'
    to_dir=f'{path}/{bird_class}'
    os.makedirs(dir, exist_ok=True)# Создаем папку вида птиц/male
    download_images(dir, urls=search_images(f'{birds_dict[bird_class][-1]} female',20))# Грузим просто птиц, для референса
    species_name=birds_dict[bird_class][0]
    species_name=''.join(species_name.replace(' ','_').split(','))#удаляем запятые и заменяем пробеля на _
    prefix=f'00000_{species_name}_реф_'## ноли в начале, для того, чтобы в папке по алфавиту быть на верху
    rename_files_in_dir(dir,prefix)# переименовываем файлы в папке     
    move_files(dir,to_dir)# перемещаем файлы в родительскую папку и удаляем подпапку  
    time.sleep(10)  # Задержка на 10 секунд

    
    # Грузим по 1000 фотографий с запросами на русском и на латыни  
    for term in birds_dict[bird_class]:
        time.sleep(10)  # Задержка на 10 секунд

        if term[1] in kirilic:
            download_images(f'{path}/{bird_class}', urls=search_images(f'{term} птица фото',1000))
        else:    
            download_images(f'{path}/{bird_class}', urls=search_images(f'{term} photo',1000))


    # Ресайзим фотографии в папке. Пока предварительно. В основном, чтобы датает не раздулся до непомерных размеров.   
    resize_images(f'{path}/{bird_class}', max_size=400, dest=f'{path}/{bird_class}')
    print(f'Закончил скачивать в папку, ушло {time.time()-start_dounload }')
    with open(log, 'a+') as f:
        f.write(f'Закончил скачивать в папку, ушло {time.time()-start_dounload }\n')

    
    # проверяем все ли файлы читаются
    print(f'Проверяем, все ли файлы читаются',end=" ")
    start_ver = time.time()
    tot,delited=verify_files(f'{path}/{bird_class}')
    print(f'Всео файлов {tot}, из них удалено {delited}. ушло {time.time()-start_ver}',end="\n")
    with open(log, 'a+') as f:
        f.write(f'Всео файлов {tot}, из них удалено так как не читались {delited}. ушло {time.time()-start_ver}\n')


    # Сохраняем копию словаря минус обработаный элемент
    birds_dict_copy.pop(bird_class)
    with open('russian_birds_1.json', 'w') as f:
        json.dump(birds_dict_copy, f) # Записываем остаток словаря в файл, чтобы в случая сбоя начать с него
    print(f'Итого ушло на папку: {time.time()-start_dounload }')
    with open(log, 'a+') as f:
        f.write(f'Итого ушло на папку: {time.time()-start_dounload }\n')
    
print(f'Всего ушло времени {time.time()-start}',end="\n")    
with open(log, 'a+') as f:
    f.write(f'---------------Finish at: {datetime.datetime.now()} ---------------------\n')











0  ['Дикуша', 'Falcipennis falcipennis']
Searching for 'Falcipennis falcipennis male'
Searching for 'Falcipennis falcipennis female'
Searching for 'Falcipennis falcipennis female'
ошибка при удалении папки D:/Russian_birds_selected/0/female
Searching for 'Дикуша птица фото'
Searching for 'Falcipennis falcipennis photo'
Закончил скачивать в папку, ушло 244.12257409095764
Всео файлов 1120, из них удалено 1. ушло 14.318225383758545
Итого ушло на папку: 258.44479966163635
40  ['Свиязь, или свияга', 'Anas penelope']
Searching for 'Anas penelope male'
Searching for 'Anas penelope female'
Searching for 'Anas penelope female'
Searching for 'Свиязь, или свияга птица фото'
Searching for 'Anas penelope photo'
Закончил скачивать в папку, ушло 284.72335290908813
Всео файлов 1106, из них удалено 6. ушло 15.323848009109497
Итого ушло на папку: 300.0501801967621
80  ['Чешуйчатый крохаль', 'Mergus squamatus']
Searching for 'Mergus squamatus male'
Searching for 'Mergus squamatus female'
Searching for 'M